In [6]:
import numpy as np
import trimesh
from glob import glob
import pickle
import pandas as pd

In [4]:
# Directory for .obj files
objs = sorted(glob('pifuhd/res/pifuhd_final/recon/*.obj'))

In [5]:
# Height Predictions from Deep Network
dct = pickle.load(open('test_height_pred.pkl', 'rb'))
pred_ws = {}

for obj in objs:
    
    # Load .obj with Trimesh
    mesh = trimesh.load(obj)
    
    key = obj.split('/')[-1].split('_')[1].replace('-F', '')
    
    if key in dct:
        
        # Find the volume of bounding sphere of the object
        sphere_v = mesh.bounding_sphere.volume
        
        obj_height = ((sphere_v/((4/3)*np.pi))**(1/3))*2*100 # in m
        pred_height = dct[key]
        
        # Scale the mesh
        mesh.apply_scale(pred_height/obj_height)
        
        # Get the volume of mesh
        pred_w = mesh.volume*1000
        
        pred_ws[key] = pred_w
    
    else:
        print(key)

010101023146
01010517334
010106213117
02010512337
02020614305
02020615319
02020821322


In [10]:
# Real weight values

test_set = pd.read_csv('csvs/REGRESSION_TEST.csv', converters={'name': lambda x: str(x)})
real_ws = dict(test_set[['name', 'Weight_kg']].values)

In [11]:
# Test set Error

err = 0

for p in pred_ws:
    err += abs(pred_ws[p] - real_ws[p])
    
print("Test set error: {:.2f} kg".format(err/len(pred_ws)))

Test set error: 1.79 kg


In [12]:
print('Test set std.: %f' % np.std(list(real_ws.values())))

Test set std.: 1.987285


In [17]:
# Check mesh volume
mesh.volume # in m^3

# Scale the mesh
mesh.apply_scale()

0.0001129127831716667

In [14]:
# Visualize the mesh
# mesh.show()